In [1]:
import tensorflow as tf
import os

In [2]:
#Avoid OOm errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Removing dodgy images

In [3]:
import cv2
import imghdr

In [4]:
data_dir = 'data'
image_exts = ['jpeg','jpg','bmp','png']

In [5]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("issue with image {}".format((image_path)))

# Loading the dataset

In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 157 files belonging to 2 classes.


In [8]:
data_iterator = data.as_numpy_iterator()

In [9]:
batch = data_iterator.next()

In [10]:
#Images represented as numpy arrays
batch[0].shape

(32, 256, 256, 3)

In [11]:
#Image labels
batch[1]

array([1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1])

# 2. Preprocesing the data

## 2.1 Scale Data

In [12]:
data = data.map(lambda x,y :(x/255,y))

In [13]:
scaled_iterator = data.as_numpy_iterator()

In [14]:
batch = scaled_iterator.next()

## 2.2 Split the data

In [15]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [16]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size).skip(val_size).take(test_size)

# 3. Deep Learning Model

## 3.1 Building the deep learning model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [18]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

## 3.2 Training the model

In [21]:
logdir='logs'

In [22]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [23]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
3/3 [==============================] - 9s 2s/step - loss: 1.1402 - accuracy: 0.5521 - val_loss: 1.0236 - val_accuracy: 0.4062
Epoch 2/20
3/3 [==============================] - 5s 1s/step - loss: 0.7974 - accuracy: 0.5000 - val_loss: 0.5475 - val_accuracy: 0.6562
Epoch 3/20
3/3 [==============================] - 5s 2s/step - loss: 0.5830 - accuracy: 0.6042 - val_loss: 0.5514 - val_accuracy: 0.6250
Epoch 4/20
3/3 [==============================] - 6s 2s/step - loss: 0.5643 - accuracy: 0.7083 - val_loss: 0.4748 - val_accuracy: 0.8125
Epoch 5/20
3/3 [==============================] - 5s 2s/step - loss: 0.5183 - accuracy: 0.8438 - val_loss: 0.5502 - val_accuracy: 0.8125
Epoch 6/20
3/3 [==============================] - 6s 2s/step - loss: 0.4325 - accuracy: 0.8333 - val_loss: 0.4708 - val_accuracy: 0.7812
Epoch 7/20
3/3 [==============================] - 6s 2s/step - loss: 0.4508 - accuracy: 0.8229 - val_loss: 0.4824 - val_accuracy: 0.7812
Epoch 8/20
3/3 [=========================

## 3.3 Performance Plot

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Evaluate Performance

In [24]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [25]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [26]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [27]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.0, Recall: 0.0, Accuracy: 0.0


## Test

In [28]:
img = cv2.imread('test_happy.jpeg')
# plt.imshow(img)
# plt.show()

In [29]:
resize = tf.image.resize(img, (256,256))
# plt.imshow(resize.numpy().astype(int))
# plt.show()

In [30]:
resize.shape

TensorShape([256, 256, 3])

In [31]:
import numpy as np
np.expand_dims(resize/255, 0).shape

(1, 256, 256, 3)

In [32]:
yhat = model.predict(np.expand_dims(resize/255, 0))

1/1 [==============================] - 1s 573ms/step


In [33]:
yhat

array([[0.44151324]], dtype=float32)

In [34]:
if yhat > 0.5:
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

Predicted class is Happy


# 5. Save the Model